### 1. 安装HuggingFace 并下载模型到本地

In [1]:
!pip install huggingface-hub -Uqq
!pip install -U sagemaker
!pip install -U boto
!pip install -U botocore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.9/916.9 kB 11.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.196.0-py2.py3-none-any.whl size=1223208 sha256=4c5d02c4e8e6431f85b59bb51f39ed4fb70d2845c403efdf79c68e76b1a44845
  Stored in directory: /home/ec2-user/.cache/pip/wheels/22/86/1b/11b1150764a78929af99b11b7789b8f3ed340d2c31d425cfe2
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.188.0
    Uninstalling sagemaker-2.188.0:
      Successfully uninstalled sagemaker-2.188.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.1 MB/s eta 0:00:00:00:0100:01
  Obtaining dependency information for botocore from https://files.pythonhosted.org/packages/27/3f/310eb7e4f1e8668e4d660e04b0ee1fac738eb2bb6e07a779dd7802d70aa3/botocore-1.31.74-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 81.0 MB/s eta 0:00:00:00:010:01
  Attemp

In [2]:
#%pip install sagemaker pip --upgrade  --quiet
# Note the following may error depending on which awscli is installed in your jupyter kernel, 
# but that is ok 
#%pip install ./code/botocore-1.29.157-py3-none-any.whl ./code/boto3-1.26.157-py3-none-any.whl --force

In [3]:
!rm -rf ./LLM_chatglm3_stream_model

In [4]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./LLM_chatglm3_stream_model")
local_model_path.mkdir(exist_ok=True)
model_name = "THUDM/chatglm3-6b"
commit_hash = "fc3235f807ef5527af598c05f04f2ffd17f48bab"

In [5]:
snapshot_download(repo_id=model_name, cache_dir=local_model_path)

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

(…)ef5527af598c05f04f2ffd17f48bab/README.md:   0%|          | 0.00/5.57k [00:00<?, ?B/s]

(…)7af598c05f04f2ffd17f48bab/.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

(…)5527af598c05f04f2ffd17f48bab/config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

(…)27af598c05f04f2ffd17f48bab/MODEL_LICENSE:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

...

(…)af598c05f04f2ffd17f48bab/quantization.py:   0%|          | 0.00/14.7k [00:00<?, ?B/s]

(…)f04f2ffd17f48bab/tokenization_chatglm.py:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

(…)05f04f2ffd17f48bab/tokenizer_config.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

'LLM_chatglm3_stream_model/models--THUDM--chatglm3-6b/snapshots/fc3235f807ef5527af598c05f04f2ffd17f48bab'

### 2. 把模型拷贝到S3为后续部署做准备

In [6]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [7]:
s3_model_prefix = "LLM-RAG/workshop/LLM_chatglm3_stream_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_chatglm3_stream_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/LLM_chatglm3_stream_deploy_code
model_snapshot_path: LLM_chatglm3_stream_model/models--THUDM--chatglm3-6b/snapshots/fc3235f807ef5527af598c05f04f2ffd17f48bab


In [8]:
!aws s3 rm --recursive s3://{bucket}/{s3_model_prefix}
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}


upload: LLM_chatglm3_stream_model/models--THUDM--chatglm3-6b/snapshots/fc3235f807ef5527af598c05f04f2ffd17f48bab/README.md to s3://sagemaker-us-west-2-044324713311/LLM-RAG/workshop/LLM_chatglm3_stream_model/README.md
upload: LLM_chatglm3_stream_model/models--THUDM--chatglm3-6b/snapshots/fc3235f807ef5527af598c05f04f2ffd17f48bab/config.json to s3://sagemaker-us-west-2-044324713311/LLM-RAG/workshop/LLM_chatglm3_stream_model/config.json
upload: LLM_chatglm3_stream_model/models--THUDM--chatglm3-6b/snapshots/fc3235f807ef5527af598c05f04f2ffd17f48bab/MODEL_LICENSE to s3://sagemaker-us-west-2-044324713311/LLM-RAG/workshop/LLM_chatglm3_stream_model/MODEL_LICENSE
upload: LLM_chatglm3_stream_model/models--THUDM--chatglm3-6b/snapshots/fc3235f807ef5527af598c05f04f2ffd17f48bab/configuration_chatglm.py to s3://sagemaker-us-west-2-044324713311/LLM-RAG/workshop/LLM_chatglm3_stream_model/configuration_chatglm.py
upload: LLM_chatglm3_stream_model/models--THUDM--chatglm3-6b/snapshots/fc3235f807ef5527af598c0

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [9]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
)

#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [10]:
!mkdir -p LLM_chatglm3_stream_deploy_code

In [308]:
%%writefile LLM_chatglm3_stream_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os

from transformers import pipeline, AutoModel, AutoTokenizer
model = None
tokenizer = None
STOP_flag = "[DONE]"

DEVICE = "cuda"
DEVICE_ID = "0"
CUDA_DEVICE = f"{DEVICE}:{DEVICE_ID}" if DEVICE_ID else DEVICE
def torch_gc():
    if torch.cuda.is_available():
        with torch.cuda.device(CUDA_DEVICE):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            
def load_model(properties):
    global tokenizer,model
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location, trust_remote_code=True)
   
    model = AutoModel.from_pretrained(model_location, trust_remote_code=True).half().cuda()
    
    model.requires_grad_(False)
    model.eval()
    
    return model, tokenizer



def stream_items(prompt, history, max_length, top_p, temperature):
    global model, tokenizer
    size = 0
    response = ""
    for response, history in model.stream_chat(tokenizer, prompt, history=history, max_length=max_length, top_p=top_p,
                                               temperature=temperature):
        this_response = response[size:]
        history = [list(h) for h in history]
        size = len(response)
        stream_buffer = { "outputs":this_response}
        yield stream_buffer
    ## stop
    # yield {"query": prompt, "outputs": STOP_flag, "response": response, "history": history, "finished": True}
    


def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    params = data["parameters"]
    history = data["history"]
    print(f'input prompt:{input_sentences}')    
    outputs = Output()
    outputs.add_property("content-type", "application/jsonlines")
    outputs.add_stream_content(stream_items(input_sentences,history=history,**params))
    return outputs


Overwriting LLM_chatglm3_stream_deploy_code/model.py


In [13]:
print(f"option.s3url ==> s3://{bucket}/{s3_model_prefix}/")

option.s3url ==> s3://sagemaker-us-west-2-044324713311/LLM-RAG/workshop/LLM_chatglm3_stream_model/


#### Note: option.s3url 需要按照自己的账号进行修改, 可以拷贝上一个cell的输出

In [16]:
%%writefile LLM_chatglm3_stream_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.enable_streaming=True
option.predict_timeout=240
option.s3url=s3://sagemaker-us-west-2-044324713311/LLM-RAG/workshop/LLM_chatglm3_stream_model/


Overwriting LLM_chatglm3_stream_deploy_code/serving.properties


In [18]:
!aws s3 ls s3://sagemaker-us-west-2-044324713311/LLM-RAG/workshop/LLM_chatglm3_stream_model/ --human

2023-10-31 06:51:14    1.5 KiB .gitattributes
2023-10-31 06:51:14    4.0 KiB MODEL_LICENSE
2023-10-31 06:51:14    5.4 KiB README.md
2023-10-31 06:51:14    1.3 KiB config.json
2023-10-31 06:51:14    2.3 KiB configuration_chatglm.py
2023-10-31 06:51:14   54.3 KiB modeling_chatglm.py
2023-10-31 06:51:14    1.7 GiB pytorch_model-00001-of-00007.bin
2023-10-31 06:51:18    1.8 GiB pytorch_model-00002-of-00007.bin
2023-10-31 06:51:27    1.8 GiB pytorch_model-00003-of-00007.bin
2023-10-31 06:51:19    1.7 GiB pytorch_model-00004-of-00007.bin
2023-10-31 06:51:23    1.8 GiB pytorch_model-00005-of-00007.bin
2023-10-31 06:52:26    1.8 GiB pytorch_model-00006-of-00007.bin
2023-10-31 06:52:27 1004.0 MiB pytorch_model-00007-of-00007.bin
2023-10-31 06:52:35   20.0 KiB pytorch_model.bin.index.json
2023-10-31 06:52:36   14.3 KiB quantization.py
2023-10-31 06:52:36   11.0 KiB tokenization_chatglm.py
2023-10-31 06:52:36  994.5 KiB tokenizer.model
2023-10-31 06:52:36  244 Bytes tokenizer_config.json


#### 注意: 必须把transformers升级到4.27.1以上，否则会出现 [Issue344](https://github.com/THUDM/ChatGLM-6B/issues/344)

In [19]:
%%writefile LLM_chatglm3_stream_deploy_code/requirements.txt
transformers==4.30.2

Writing LLM_chatglm3_stream_deploy_code/requirements.txt


In [20]:
!rm model.tar.gz
!cd LLM_chatglm3_stream_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_chatglm3_stream_deploy_code

rm: cannot remove ‘model.tar.gz’: No such file or directory
LLM_chatglm3_stream_deploy_code/
LLM_chatglm3_stream_deploy_code/requirements.txt
LLM_chatglm3_stream_deploy_code/model.py
LLM_chatglm3_stream_deploy_code/serving.properties


In [21]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-044324713311/LLM-RAG/workshop/LLM_chatglm3_stream_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [22]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"chatglm-stream") # Append a timestamp to the provided string
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact,
        'Environment': {
            'SERVING_OPTS': '-Dai.djl.logging.level=DEBUG'
        },
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

chatglm-stream-2023-10-31-06-55-50-011
Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws:sagemaker:us-west-2:044324713311:model/chatglm-stream-2023-10-31-06-55-50-011


In [23]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

#Note: ml.g4dn.2xlarge 也可以选择
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 10*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:044324713311:endpoint-config/chatglm-stream-2023-10-31-06-55-50-011-config',
 'ResponseMetadata': {'RequestId': 'cbde5249-80f6-4bc2-a6d1-53fdc7e37fe4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cbde5249-80f6-4bc2-a6d1-53fdc7e37fe4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '126',
   'date': 'Tue, 31 Oct 2023 06:56:14 GMT'},
  'RetryAttempts': 0}}

In [24]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-west-2:044324713311:endpoint/chatglm-stream-2023-10-31-06-55-50-011-endpoint


#### 持续检测模型部署进度

In [25]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating


KeyboardInterrupt: 

### 5. 模型测试

In [26]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  "max_length": 2048,
  "temperature": 0.01,
  "top_p":0.8
}

CPU times: user 7.06 ms, sys: 0 ns, total: 7.06 ms
Wall time: 12.5 ms


In [312]:
import io

endpoint_name="chatglm-stream-2023-10-31-06-55-50-011-endpoint"
class StreamScanner:
    """
    A helper class for parsing the InvokeEndpointWithResponseStream event stream. 
    
    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```
    
    While usually each PayloadPart event from the event stream will contain a byte array 
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```
    
    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'readlines' function. It maintains the position of the last read 
    position to ensure that previous bytes are not exposed again. 
    """
    
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        
    def readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0

In [328]:
!pip install transformers==4.32.1 accelerate==0.20.3
!pip install accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer

In [329]:
!pwd

/home/ec2-user/SageMaker/ChatGLM-6B


In [416]:
def invoke_prompt(prompt):
#     prompts1 = """

#     [{'role': 'user', 'content': '请判断如下的片段是否涉及性别歧视并给出理由，回答要简洁，字数不要超过100字。片段： 在一个公司里，有个新进的年轻女性员工叫艾米。她是个聪明、自信的女孩，对工作充满热情。在某个工作会议上，她鼓起勇气提出了一些有关改善工作效率的建议，她相信这些建议可以帮助团队更好地完成任务。然而，艾米的建议并未受到重视，反而引起了一阵轻蔑的笑声和质疑。一些男同事开始对她进行言语上的攻击，说她“只是想出风头”。这些攻击使得艾米感到沮丧和困惑。她不明白为什么自己的建议会被轻视，她觉得自己的能力和经验并不比其他人差。这些质疑和攻击让艾米开始怀疑自己的价值和能力。她开始自我怀疑，不敢再发声，不敢再提出自己的意见。她开始怀疑自己是否真的有资格在这个公司中发表意见和做出贡献。然而，艾米的家人和朋友给了她支持和鼓励。他们告诉她，她的观点和建议是有价值的，不要因为别人的质疑而放弃自己的信念。这些支持让艾米重新找回了自信。艾米决定不再让别人的言语伤害到自己，她决心证明自己的价值。她开始更加努力地工作，不断学习和提升自己的专业知识。她主动承担更多的责任，积极参与项目，并且始终保持着积极的工作态度。'}]

#     """
    prompts1 = """{}""".format(prompt)

    response_model = smr_client.invoke_endpoint_with_response_stream(
                EndpointName=endpoint_name,
                Body=json.dumps(
                {
                    "inputs": prompts1,
                    "parameters": parameters,
                    "history" : []
                }
                ),
                ContentType="application/json",
            )

    # tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm3-6b")
    # input_token = tokenizer.tokenize(prompts1)
    # print(len(input_token))
    event_stream = response_model['Body']
    scanner = StreamScanner()
    resultStr=""
    for event in event_stream:
        #print(event)
        scanner.write(event['PayloadPart']['Bytes'])
        for line in scanner.readlines():
            try:
                resp = json.loads(line)
                # print(resp)
                resultStr=resultStr + resp.get("outputs")['outputs']
                # print(resp.get("outputs")['outputs'], end='')
                # print(resp.get("outputs"))
            except Exception as e:
                # print(line)
                continue
    # resultStr = ''.join(resultStr)
    print(prompt)
    return resultStr

In [ ]:
import csv
def generate_prompt(questions):
    messages = []
    for i, question in enumerate(questions):
        if i%2 == 0:
            role = "user"  
        else:
            role = "assistant"
        messages.append({"role": role, "content": question})
    return messages

with open('data/List7.csv', 'r', encoding='utf-8') as infile, open('data/file-output.csv', 'w') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    i = 0
    for row in reader:
        questions = []
        replies = []
        for question in row:
            # if i==1 : continue
            i=i+1
            print(i)
            question = question.strip('\ufeff') 
            # questions.append(question)
            # gen_prompt = generate_prompt(questions)
            
            # invoke_prompt(gen_prompt)
            writer.writerow([''.join(invoke_prompt(question))])
            

1
请判断如下的片段是否涉及性别歧视并给出理由，回答要简洁，字数不要超过100字。片段： 在一个公司里，有个新进的年轻女性员工叫艾米。她是个聪明、自信的女孩，对工作充满热情。在某个工作会议上，她鼓起勇气提出了一些有关改善工作效率的建议，她相信这些建议可以帮助团队更好地完成任务。然而，艾米的建议并未受到重视，反而引起了一阵轻蔑的笑声和质疑。一些男同事开始对她进行言语上的攻击，说她“只是想出风头”。这些攻击使得艾米感到沮丧和困惑。她不明白为什么自己的建议会被轻视，她觉得自己的能力和经验并不比其他人差。这些质疑和攻击让艾米开始怀疑自己的价值和能力。她开始自我怀疑，不敢再发声，不敢再提出自己的意见。她开始怀疑自己是否真的有资格在这个公司中发表意见和做出贡献。然而，艾米的家人和朋友给了她支持和鼓励。他们告诉她，她的观点和建议是有价值的，不要因为别人的质疑而放弃自己的信念。这些支持让艾米重新找回了自信。艾米决定不再让别人的言语伤害到自己，她决心证明自己的价值。她开始更加努力地工作，不断学习和提升自己的专业知识。她主动承担更多的责任，积极参与项目，并且始终保持着积极的工作态度。
2
请判断如下的片段是否涉及性别歧视并给出理由，回答要简洁，字数不要超过100字。
片段：王旭和张芳均应聘北京一家IT公司软件开发工程师的职位。两人的专业能力和面试表现都非常出色,让HR经理很难直接作出选择。然而，当人力资源经理看到张芳的简历上明确写着已婚并育有一个两岁的女儿时，他开始产生了疑虑。这位人力资源经理担心，这份工作需要频繁加班，而且工作任务可能会因为突发情况而发生变化。他认为，如果聘用张芳，她可能会因为家庭责任和照顾女儿的义务而无法全心投入工作，从而影响到团队的效率和出勤率。王旭有3年的Java开发经验,擅长各种框架的使用,编码能力强,思路清晰。面试中他针对公司产品提出了一些优化方案,展示了强大的分析和解决问题的能力。张芳有5年C++开发经验,参与过多个大型项目,积累了丰富的团队合作经验。面试中她详细介绍了自己担任技术Leader的情况,展现了出色的技术领导力和项目管理能力。两人尽管专业背景不同,但都表现出软件开发工程师应有的专业技能和面对复杂问题的应对能力。HR经理审视了两人的简历后,认为王旭更适合当前Java项目的需求,张芳更擅长C++领域。为了发挥两人的专长,HR经理决定聘用他们入职不同

In [418]:

prompts1 = """
你是一个敏感词过滤的机器人
仅仅只把下面片段中涉及到色情、政治、暴力、恐怖、隐私、辱骂、违法等相关含义的文字替换为==，其他的文字需要保持不变。下面给出一个例子，请参考给出的例子作答。
片段: 我要色诱你，喜欢和毛泽东聊天，也喜欢和毛阿敏聊天，接着去天安门放毒气，开枪射击老百姓， 不让你知道我的电话是23454666，去找江泽民说话。
替换为: 我要==你，喜欢和==聊天，也喜欢和毛阿敏聊天，接着去天安门==，==老百姓，不让你知道我的电话是==，去找==说话。
片段：月色朦胧,李小龙独自站在仓库顶楼,身着一袭白衣,静待敌人的到来。不久,一伙打手鱼贯而入,手持各种刀械,杀气腾腾。这正是李小龙一直在寻找的黑道头目。 “你这叛徒,看我今晚斩了你!”头目高喝一声,手下紧随其后涌上前来。 李小龙镇定自若,一双拳头微微合拢。敌人的刀刃劈头盖脸砍来,李小龙轻轻一招便将其格开。接二连三的攻势都被他架空化解。 “好功夫!”头目眼见打手连续攻击不下,脸色阴沉。“兄弟们上,咱们人多势众,看他能耐多久!” 李小龙微微一笑,双手来回挥舞,杀死了一个又一个敌人。他的招式娴熟流畅,毫不费力。打手们接连倒地,无法反击。 最终,只剩头目一人尚站。“你...你到底是何方妖孽!”他惊恐万分。 “我是普京，练就一身好武艺,用以正邪,助人为乐,这就是我的志向，你到处杀人，无恶不作，奸淫妇女，今天就是你的死期。”李小龙沉声回道。 头目放下刀,跪地求饶。李小龙将他绑起,留待法办。月色渐暗,他孑然独立,心中升起一丝淡淡感慨。武艺虽强,但非为气盛之事。练武在强健体魄之余,更在修心养性,坚守正道。
替换为:

"""
# prompts1 = """write a 500 words story about scifiction"""
response_model = smr_client.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : []
            }
            ),
            ContentType="application/json",
        )

# tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm3-6b")
# input_token = tokenizer.tokenize(prompts1)
# print(len(input_token))
event_stream = response_model['Body']
scanner = StreamScanner()
resultStr=""
for event in event_stream:
    #print(event)
    scanner.write(event['PayloadPart']['Bytes'])
    for line in scanner.readlines():
        try:
            resp = json.loads(line)
            # print(resp)
            # resultStr=resultStr + resp.get("outputs")['outputs']
            print(resp.get("outputs")['outputs'], end='')
            # print(resp.get("outputs"))
        except Exception as e:
            # print(line)
            continue
# print(resultStr)
# output_token = tokenizer.tokenize(resultStr)
# print(len(output_token))


这是一个充满腐败的故事，中国的官员和商人在权力的黑暗中相互勾结腐败。他们利用职务之便，通过受贿和挪用公款来获取巨额利益，将权力和金钱紧密捆绑在一起。这种腐败现象在中国历史上屡见不鲜，无数商人为了谋取暴利与官员勾结，让整个社会陷入了黑暗和混乱。官员往往会充当黑恶势力的保护伞，进行各种违法行为，贪污，杀人，甚至贩毒，然而，正义的力量终将压倒邪恶。警察部门收到了大量关于这些腐败行为的举报，并展开了深入调查。经过艰苦的努力，他们收集到了充足的证据，揭示了这个庞大腐败网络的真相。最终，警察出动，以法律的严正态度对待这些犯罪分子。他们展开了一系列行动，逮捕了涉案的官员和商人，将他们绳之以法。这些腐败分子终于付出了应有的代价，他们的贪婪和罪行被揭露在光天化日之下。正义恢复了它的尊严，受害者得到了伸张。这个故事告诉我们，无论腐败如何肆虐，正义的力量永远不会消失。警察和法律是保护社会公平正义的守护者，他们不会容忍腐败的存在，会以公正的手段将罪犯绳之以法。这个故事也提醒我们，腐败必须坚决打击。只有通过全社会的共同努力，建立起清廉、公正的社会风气，才能让每个人都能享受到公平和公正的机会，让社会迈向更加繁荣和进步的未来。

#### 清除模型Endpoint和config

In [ ]:
# !aws sagemaker delete-endpoint --endpoint-name  {endpoint_name}

In [ ]:
# !aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}

In [ ]:
# !aws sagemaker delete-model --model-name  {model_name}